# Scripts

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def obtener_conteido_pagina(url):
    response = requests.get(url)
    return response.text

In [3]:
def analizar_contenido_html(html):
    return BeautifulSoup(html, 'html.parser')

In [22]:
data =[]
def procesar_pagina(soup):
    titulos = []
    precios_originales = []
    precios_descuento = []
    precios_envio = []
    skus = []
    stocks = []
    
    titulo_items = soup.find_all('div','emproduct_right')
    
    for item in titulo_items:
        titulo = item.a.text.strip()
        titulos.append(titulo)
        
    precio_original_items = soup.find_all('span', class_ = 'oldPrice')
    
    for item in precio_original_items:
        precio_original = item.text.strip()
        precios_originales.append(precio_original)
        
    precio_descuento_items = soup.find_all('div', class_ = 'emproduct_right_price_left')
    
    for item in precio_descuento_items:
        precio_descuento = item.label.text.strip()
        precios_descuento.append(precio_descuento)
        
    sku_items = soup.find_all('div', class_='emproduct_right_artnum')
    
    for item in sku_items:
        sku = item.text.strip()
        skus.append(sku.lstrip('SKU: '))
        
    precio_envio_items = soup.find_all('span', class_={'deliveryvalue', 'deliverytextfree'})
    
    for item in precio_envio_items:
        precio_envio = item.text
        precios_envio.append(precio_envio)
    
    stock_items = soup.find_all('div', class_='emstock')
    
    for item in stock_items:
        stock = item.text.split()
        stocks.append(stock[1])
    
    min_length = min(len(titulos), len(skus), len(precios_originales), len(precios_descuento), len(precios_envio), len(stocks))
    for i in range(min_length):
        data.append({
            "Titulo": titulos[i] if i < len(titulos) else None,
            "Sku": skus[i] if i < len(skus) else None,
            "Precio original": precios_originales[i] if i < len(precios_originales) else None,
            "Precio Descuento": precios_descuento[i] if i < len(precios_descuento) else None,
            "Precio Envio": precios_envio[i] if i < len(precios_envio) else None,
            "Stock": stocks[i] if i < len(stocks) else None
        })
   
    
            
            


In [5]:
def manegar_paginacion(url_base, num_paginas):
    for i in range(1, num_paginas + 1):
            url = url_base + str(i)+'/'
            contenido_pagina = obtener_conteido_pagina(url)
            soup = analizar_contenido_html(contenido_pagina)
            procesar_pagina(soup)

In [23]:
url_base = "https://www.cyberpuerta.mx/Promociones/"
num_paginas = 10

manegar_paginacion(url_base, num_paginas)

In [24]:
data

[{'Titulo': 'Procesador AMD Ryzen 5 5600G con Gráficos Radeon 7, S-AM4, 3.90GHz, Six-Core, 16MB L3 Caché - incluye Disipador Wraith Stealth',
  'Sku': '100-100000252BOX',
  'Precio original': '$2,098.00',
  'Precio Descuento': '$1,819.00',
  'Precio Envio': '$99.00',
  'Stock': '97'},
 {'Titulo': 'Memoria RAM HPE DDR4, 2933MHz, 32GB, CL21, Registered (Buffered)',
  'Sku': 'P00924-B21',
  'Precio original': '$15,352.00',
  'Precio Descuento': '$2,309.00',
  'Precio Envio': '$99.00',
  'Stock': '21'},
 {'Titulo': 'SSD Adata FALCON NVMe, 1TB, PCI Express 3.0, M.2',
  'Sku': 'AFALCON-1T-C',
  'Precio original': '$1,883.00',
  'Precio Descuento': '$899.00',
  'Precio Envio': '$99.00',
  'Stock': '5'},
 {'Titulo': 'SSD Western Digital WD Blue SN570 NVMe, 500GB, PCI Express 3.0, M.2 ― Incluye Membresía 1 Mes de Adobe Creative Cloud',
  'Sku': 'WDS500G3B0C',
  'Precio original': '$694.00',
  'Precio Descuento': '$509.00',
  'Precio Envio': 'Envío gratis',
  'Stock': '16'},
 {'Titulo': 'Router 

In [25]:
import pandas as pd
df = pd.DataFrame(data)

In [31]:

import datetime

fecha_actual = datetime.datetime.now().strftime("%d-%m-%Y")
df.to_csv(f"dataset/web scraping/promociones-cuberpuerta-{fecha_actual}.csv", index=False)